In [1]:
import sys
from pennylane import numpy as np
import pennylane as qml

In [2]:
numbers=[0,0,0,1,1,0,1,1]

def f1(wires):
    qml.CNOT(wires=[wires[numbers[0]], wires[2]])
    qml.CNOT(wires=[wires[numbers[1]], wires[2]])

def f2(wires):
    qml.CNOT(wires=[wires[numbers[2]], wires[2]])
    qml.CNOT(wires=[wires[numbers[3]], wires[2]])

def f3(wires):
    qml.CNOT(wires=[wires[numbers[4]], wires[2]])
    qml.CNOT(wires=[wires[numbers[5]], wires[2]])
    qml.PauliX(wires=wires[2])

def f4(wires):
    qml.CNOT(wires=[wires[numbers[6]], wires[2]])
    qml.CNOT(wires=[wires[numbers[7]], wires[2]])
    qml.PauliX(wires=wires[2])

fs = [f1, f2, f3, f4]

In [3]:
dev1 = qml.device('default.qubit', wires=5, shots=1)
dev2 = qml.device('default.qubit', wires=3, shots=1)


@qml.qnode(dev1)
def ctrl_version_DJ(index):
    qml.PauliX(wires=4)
    [qml.Hadamard(wires=w) for w in range(2, 5)]

    qml.BasisStatePreparation(list(map(int, np.binary_repr(index, width=2))), wires=range(2))
    def temp():
        fs[index](wires=range(2, 5))
    qml.ctrl(temp, control=range(2))()
    qml.BasisStatePreparation(list(map(int, np.binary_repr(index, width=2))), wires=range(2))

    [qml.Hadamard(wires=w) for w in range(2, 4)]
    return qml.sample(wires=range(2, 4))

@qml.qnode(dev2)
def orig_version_DJ(index):
    qml.PauliX(wires=2)
    [qml.Hadamard(wires=w) for w in range(3)]

    def temp():
        fs[index](wires=range(3))
    temp()

    [qml.Hadamard(wires=w) for w in range(2)]
    return qml.sample(wires=range(2))



In [4]:

def checker(sample):
    for s in sample:
        if s==1:
            return 'balanced'
    return 'constant'


In [5]:
for index in range(4):
    print((ctrl_version_DJ(index)))
    print((orig_version_DJ(index)))
    print()

[0 0]
[0 0]

[0 0]
[1 1]

[0 0]
[1 1]

[0 0]
[0 0]

